In [2]:
import os
from typing import Literal

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.types import Command

# 初始化 LLM
load_dotenv()

# --- 初始化模型，并绑定工具 ---
os.environ['DEEPSEEK_API_KEY'] = os.getenv("DEEPSEEK_API_KEY")
llm = ChatOpenAI(
    model="deepseek-chat",  # DeepSeek的聊天模型
    temperature=0,
    openai_api_key=os.getenv("DEEPSEEK_API_KEY"),  # 从DeepSeek获取的API key
    openai_api_base="https://api.deepseek.com/v1"  # DeepSeek的Base URL
)


In [3]:
# 定义两个 agent 的行为函数
def agent_A(state: MessagesState) -> Command[Literal["agent_B", END]]:
    msgs = state["messages"]
    last = msgs[-1].content if msgs else ""
    if "数学" in last:
        # 转到 agent_B
        return Command(goto="agent_B", update={})
    else:
        resp = llm.invoke(msgs + [("assistant", "Agent A 回答: “非数学问题已处理”")])
        # 更新 state.messages
        return Command(
            goto=END,
            update={"messages": state["messages"] + [{"role":"assistant", "content": resp.content}]}
        )

def agent_B(state: MessagesState) -> Command[Literal["agent_A", END]]:
    msgs = state["messages"]
    resp = llm.invoke(msgs + [("assistant", "Agent B 回答: “这是数学问题的解答部分”")])
    return Command(
        goto="agent_A",
        update={"messages": state["messages"] + [{"role":"assistant", "content": resp.content}]}
    )

In [4]:
# 构建 StateGraph
graph_builder = StateGraph(MessagesState)

# 添加节点
graph_builder.add_node("agent_A", agent_A)
graph_builder.add_node("agent_B", agent_B)

# 设置入口
# 方法一: set entry point
graph_builder.set_entry_point("agent_A")

# 添加边（控制流）
graph_builder.add_edge("agent_A", "agent_B")
graph_builder.add_edge("agent_A", END)
graph_builder.add_edge("agent_B", "agent_A")

# 编译 graph
compiled_graph = graph_builder.compile()


In [6]:
from IPython.display import Image, display

display(Image(compiled_graph.get_graph(xray=True).draw_mermaid_png(max_retries=5, retry_delay=2.0)))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`